<a href="https://colab.research.google.com/github/Perfect-Cube/Smart-India-Hackathon-24/blob/main/Kalman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class KalmanFilter:
    def __init__(self, dt, u_x, u_y, std_acc, std_meas):
        # Define sampling time
        self.dt = dt

        # Define the control input variables (external motion)
        self.u = np.matrix([[u_x], [u_y]])

        # Initial state vector [x_position, y_position, x_velocity, y_velocity]
        self.x = np.matrix([[0], [0], [0], [0]])

        # Define the state transition matrix A
        self.A = np.matrix([[1, 0, self.dt, 0],
                            [0, 1, 0, self.dt],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])

        # Define the control input matrix B
        self.B = np.matrix([[(self.dt**2) / 2, 0],
                            [0, (self.dt**2) / 2],
                            [self.dt, 0],
                            [0, self.dt]])

        # Define measurement matrix H
        self.H = np.matrix([[1, 0, 0, 0],
                            [0, 1, 0, 0]])

        # Define process noise covariance matrix Q
        self.Q = np.matrix([[self.dt**4 / 4, 0, self.dt**3 / 2, 0],
                            [0, self.dt**4 / 4, 0, self.dt**3 / 2],
                            [self.dt**3 / 2, 0, self.dt**2, 0],
                            [0, self.dt**3 / 2, 0, self.dt**2]]) * std_acc**2

        # Define measurement noise covariance matrix R
        self.R = np.matrix([[std_meas**2, 0],
                            [0, std_meas**2]])

        # Define the error covariance matrix P
        self.P = np.eye(self.A.shape[1])

    def predict(self):
        # Predicted state estimate
        self.x = np.dot(self.A, self.x) + np.dot(self.B, self.u)

        # Predicted covariance estimate
        self.P = np.dot(self.A, np.dot(self.P, self.A.T)) + self.Q

        return self.x[0:2]

    def update(self, z):
        # Measurement residual
        y = z - np.dot(self.H, self.x)

        # Residual covariance
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        # Kalman gain
        K = np.dot(self.P, np.dot(self.H.T, np.linalg.inv(S)))

        # Updated state estimate
        self.x = self.x + np.dot(K, y)

        # Updated covariance estimate
        I = np.eye(self.H.shape[1])
        self.P = (I - np.dot(K, self.H)) * self.P

        return self.x[0:2]

# Example of using the KalmanFilter class for object tracking

dt = 1.0  # Time step
u_x, u_y = 0.0, 0.0  # Control input (acceleration)
std_acc = 0.5  # Standard deviation of the acceleration (process noise)
std_meas = 1.0  # Standard deviation of the measurement noise

# Initialize the Kalman Filter
kf = KalmanFilter(dt, u_x, u_y, std_acc, std_meas)

# Simulated noisy measurements (positions)
measurements = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])

for measurement in measurements:
    predicted = kf.predict()
    updated = kf.update(np.matrix(measurement).T)

    print(f"Predicted: {predicted.T}")
    print(f"Updated: {updated.T}")


Predicted: [[0. 0.]]
Updated: [[0.67346939 1.34693878]]
Predicted: [[1.04081633 2.08163265]]
Updated: [[1.70998843 2.72232935]]
Predicted: [[2.46278442 3.82607019]]
Updated: [[2.82762168 3.94419051]]
Predicted: [[3.76649884 5.1081773 ]]
Updated: [[3.91844504 5.03778309]]
Predicted: [[4.93073038 6.16776106]]
Updated: [[4.97469826 6.06127719]]


In [3]:
import numpy as np
import folium

# KalmanFilter class from previous code (paste here)

# Generate some simulated noisy GPS data for a vehicle
def generate_noisy_gps_data(true_path, noise_std):
    noisy_path = []
    for point in true_path:
        noisy_point = point + np.random.normal(0, noise_std, point.shape)
        noisy_path.append(noisy_point)
    return np.array(noisy_path)

# Initialize Kalman Filter and Map
def map_vehicle_path(true_path, noisy_path, std_acc, std_meas):
    # Initialize Kalman Filter
    dt = 1.0
    kf = KalmanFilter(dt, 0.0, 0.0, std_acc, std_meas)

    # Create a folium map centered on the initial position
    start_lat, start_lon = true_path[0]
    vehicle_map = folium.Map(location=[start_lat, start_lon], zoom_start=15)

    # Add the true path (for comparison)
    folium.PolyLine(true_path, color='green', weight=2.5, opacity=0.8).add_to(vehicle_map)

    # Add the noisy path (before Kalman filter)
    folium.PolyLine(noisy_path, color='red', weight=2.5, opacity=0.6, dash_array='5').add_to(vehicle_map)

    # Add optimized path after Kalman Filter
    optimized_path = []

    for i, measurement in enumerate(noisy_path):
        kf.predict()
        optimized_position = kf.update(np.matrix(measurement).T)
        optimized_path.append(optimized_position.flatten().tolist()[0])

    # Plot the optimized path
    folium.PolyLine(optimized_path, color='blue', weight=2.5, opacity=0.8).add_to(vehicle_map)

    # Save map to HTML file
    vehicle_map.save('vehicle_path_map.html')
    print("Map saved to 'vehicle_path_map.html'")

# Simulated true path of the vehicle (lat, lon)
true_path = np.array([
    [37.7749, -122.4194],  # San Francisco (start)
    [37.7750, -122.4180],
    [37.7755, -122.4170],
    [37.7760, -122.4160],
    [37.7765, -122.4150],  # (end point)
])

# Generate noisy GPS data
noise_std = 0.001  # Standard deviation for GPS noise
noisy_path = generate_noisy_gps_data(true_path, noise_std)

# Use Kalman filter to smooth the noisy path and plot it
map_vehicle_path(true_path, noisy_path, std_acc=0.1, std_meas=0.1)

Map saved to 'vehicle_path_map.html'
